<a href="https://colab.research.google.com/github/RaviKcse08/datascience_projs/blob/main/DC_Intro_std_mutualInformation.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from pprint import pprint

In [4]:

#read bank marketing csv
sd = pd.read_csv('bank_marketing.csv')

sd.head()




,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,,pdays,previous,putcome,y
0,51,management,married,tertiary,no,6530,yes,no,unknown,5,may,91,1,-1,0,unknown,no
1,27,unemployed,married,primary,no,7459,yes,no,unknown,9,may,407,2,-1,0,unknown,no
2,59,management,divorced,primary,no,13308,yes,no,unknown,13,may,131,1,-1,0,unknown,no
3,48,technician,married,secondary,no,7373,yes,no,unknown,14,may,126,1,-1,0,unknown,no
4,51,blue-collar,married,secondary,no,7180,yes,no,unknown,14,may,927,1,-1,0,unknown,yes


In [5]:
#check Unique values of education feature
print(sd['education'].unique())


['tertiary' 'primary' 'secondary' 'unknown']


In [6]:
 #transform education feature into categorical classes
edn = {'tertiary': 101, 'primary': 102, 'secondary': 103, 'unknown': 104}

sd['edcn'] = sd['education'].map(edn)
sd.head()



,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,,pdays,previous,putcome,y,edcn
0,51,management,married,tertiary,no,6530,yes,no,unknown,5,may,91,1,-1,0,unknown,no,101
1,27,unemployed,married,primary,no,7459,yes,no,unknown,9,may,407,2,-1,0,unknown,no,102
2,59,management,divorced,primary,no,13308,yes,no,unknown,13,may,131,1,-1,0,unknown,no,102
3,48,technician,married,secondary,no,7373,yes,no,unknown,14,may,126,1,-1,0,unknown,no,103
4,51,blue-collar,married,secondary,no,7180,yes,no,unknown,14,may,927,1,-1,0,unknown,yes,103


In [7]:
#check unique values of marital feature
print(sd['marital'].unique())

['married' 'divorced' 'single']


In [8]:
 #transform marital status feature into categorical classes
mrtl = {'married': 201, 'divorced': 202, 'single': 203}

sd['mrtal'] = sd['marital'].map(mrtl)
sd.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,,pdays,previous,putcome,y,edcn,mrtal
0,51,management,married,tertiary,no,6530,yes,no,unknown,5,may,91,1,-1,0,unknown,no,101,201
1,27,unemployed,married,primary,no,7459,yes,no,unknown,9,may,407,2,-1,0,unknown,no,102,201
2,59,management,divorced,primary,no,13308,yes,no,unknown,13,may,131,1,-1,0,unknown,no,102,202
3,48,technician,married,secondary,no,7373,yes,no,unknown,14,may,126,1,-1,0,unknown,no,103,201
4,51,blue-collar,married,secondary,no,7180,yes,no,unknown,14,may,927,1,-1,0,unknown,yes,103,201


In [68]:
#Select Xs and Y
df = sd.rename(columns={"y": "label"})


##Select Xs - age, job, marital, education, balance)
#print(sd['education'].unique())
df = df[['age','edcn','mrtal','balance','label']].head(5000)
df.shape


#taking 500 records and 5 features

(5000, 5)

In [69]:
train_df, test_df = train_test_split(df, test_size=.20)
train_df.shape

data = train_df.values
data[:5]

#split data with train_test_split function based on 20% ratio
#data[:5]

array([[55, 103, 201, 1105, 'no'],
       [40, 103, 203, 737, 'no'],
       [48, 102, 201, 231, 'no'],
       [40, 102, 201, 1758, 'yes'],
       [24, 102, 201, -427, 'no']], dtype=object)

In [70]:
#helper function to check uncertainity
def calculate_uncertainity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False
        

In [71]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [72]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [73]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [74]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [105]:
def calculate_mutual_information(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    sum_mutual_information =  calculate_entropy(data_below) + calculate_entropy(data_above) + (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    std_mutual_information = sum_mutual_information / calculate_entropy(data_below)
    
    return std_mutual_information

In [106]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_mutual_information(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [107]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [108]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (calculate_uncertainity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # checking for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [109]:
tree = decision_tree_algorithm(train_df, max_depth=9)
pprint(tree)

<ipython-input-105-fbaf06058cd6>:9: RuntimeWarning: divide by zero encountered in double_scalars
  std_mutual_information = sum_mutual_information / calculate_entropy(data_below)


{'age <= 72': [{'balance <= 45248': [{'balance <= 37378': [{'balance <= 14220': [{'age <= 71': [{'balance <= 13851': [{'balance <= 13711': [{'age <= 69': [{'age <= 67': ['no',
                                                                                                                                                                          'yes']},
                                                                                                                                                           'no']},
                                                                                                                                            'no']},
                                                                                                                      'yes']},
                                                                                                'no']},
                                                                                 'no']},
              

In [110]:
def classify_sample(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":  # feature is continuous
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_sample(example, residual_tree)

In [111]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_sample, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [112]:
accuracy = calculate_accuracy(test_df, tree)
print(accuracy*100)

95.6
